In [ ]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

In [ ]:
# only once
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
# Read excel
schedules_df = pd.read_excel('/content/drive/MyDrive/Synthesis Project I/Data/Schedules.xlsx')
data_df = pd.read_excel('/content/drive/MyDrive/Synthesis Project I/Data/Data.xlsx')
clients_df = pd.read_excel('/content/drive/MyDrive/Synthesis Project I/Data/Clients.xlsx')
transl_cost_pairs_df = pd.read_excel('/content/drive/MyDrive/Synthesis Project I/Data/TranslatorsCost+Pairs.xlsx')

In [ ]:
data_df['HOURS'] = data_df['HOURS'].str.replace(',', '.').astype(float)


#TODO probablemente falten más datos que arreglar

# # Show head of each one
# print(schedules_df.head())
# print(data_df.head())
# print(clients_df.head())
# print(transl_cost_pairs_df.head())

In [16]:
# See the types of each column
print(schedules_df.dtypes)
print(data_df.dtypes)
print(clients_df.dtypes)
print(transl_cost_pairs_df.dtypes)

NAME     object
START    object
END      object
MON       int64
TUES      int64
WED       int64
THURS     int64
FRI       int64
SAT       int64
SUN       int64
dtype: object
PROJECT_ID                             object
PM                                     object
TASK_ID                                 int64
START                                  object
END                            datetime64[ns]
TASK_TYPE                              object
SOURCE_LANG                            object
TARGET_LANG                            object
TRANSLATOR                             object
ASSIGNED                       datetime64[ns]
READY                          datetime64[ns]
WORKING                        datetime64[ns]
DELIVERED                      datetime64[ns]
RECEIVED                       datetime64[ns]
CLOSE                          datetime64[ns]
HOURS                                 float64
HOURLY_RATE                             int64
COST                                  float6

What do we want?
A client will state his preferences, or his *ideal translator*.

1.   We first **filter** directly by input and output language.

2.   Then we use a **similarity** measure for quality, deadline and price.

3.   Lastly we do a **ranking** to sort the candidates by the indicated field, wildcard and of course availability.


(Probablemente me esté dejando algo, como tener en cuenta en el ranking si el cliente suele elegir un traductor específico) (?)


In [17]:
# The information we need for the similarity is a priori in these columns
# transl_cost_pairs_df['HOURLY_RATE'] + data_df['HOURS'] + data_df['QUALITY_EVALUATION']

# Add the languages, translator's names and the price (HOURLY_RATE)
df_for_knn = transl_cost_pairs_df.copy()


# QUALITY_EVALUATION and HOURS: are for a specific language and field, so we should add them to each of its possibilities

# Take the quality and HOURS for each possibility, doing a mean if there are various instances of this type of translation
data_reduced = data_df.groupby(['TRANSLATOR', 'SOURCE_LANG', 'TARGET_LANG'], as_index=False)[['QUALITY_EVALUATION', 'HOURS']].mean() #TODO gaussian mean? (requires timeline)

# Merge with df_for_knn
df_for_knn = df_for_knn.merge(data_reduced, on=['TRANSLATOR', 'SOURCE_LANG', 'TARGET_LANG'], how='left')

df_for_knn


,TRANSLATOR,SOURCE_LANG,TARGET_LANG,HOURLY_RATE,QUALITY_EVALUATION,HOURS
0,Aaron,English,Spanish (Iberian),19,5.500000,0.000000
1,Abdon Isaias,English,Catalan,16,8.000000,7.870000
2,Abdon Isaias,English,English,24,9.000000,0.500000
3,Abdon Isaias,English,Spanish (Global),13,7.046154,6.436308
4,Abdon Isaias,English,Spanish (Iberian),17,7.008239,3.509382
...,...,...,...,...,...,...
3789,Zachary,Portuguese (Brazil),English,22,6.000000,1.290000
3790,Zachary,Portuguese (SOURCE),English,16,5.000000,2.020000
3791,Zachary,Spanish (Iberian),English,24,7.125000,0.758750
3792,Zlatan,English,Swedish,40,6.818182,3.264545


In [ ]:
# Now an example of how it would be for a specific case

# Example of client
source_lang = 'English'
target_lang = 'Spanish (Iberian)'

# Filter by language
df_lang_filtered = df_for_knn[
    (df_for_knn['SOURCE_LANG'] == source_lang) &
    (df_for_knn['TARGET_LANG'] == target_lang)
].copy()


# Save the translator's name to retrieve it later
translators = df_lang_filtered[['TRANSLATOR']].copy()

# Select the numeric features for the KNN
X = df_lang_filtered[['HOURLY_RATE', 'QUALITY_EVALUATION', 'HOURS']].dropna() #TODO provisional, cómo leches tratamos a los que no han sido evaluados

# Standarize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train
knn = NearestNeighbors(n_neighbors=5, metric='euclidean')  #TODO do density area based?
knn.fit(X_scaled)

# Example of client
client = np.array([[25, 3.0, 1.5]])  # (HOURLY_RATE, QUALITY_EVALUATION, HOURS)
client_scaled = scaler.transform(client)

# Find nearest neighbours
distances, indexes = knn.kneighbors(client_scaled)

# Show recommended translators
best_translators = translators.iloc[indexes[0]]
print(best_translators) #TODO retrieve info about his quality (experience) and HOURS, as a whole? case based as we have done until now? Pensar cosas


           TRANSLATOR
543         Ana Laura
458          Ambrosia
491   Amelio Laureano
1062  Casimiro Amadeo
680   Araceli Enrique


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
